In [ ]:
import pandas as pd
from pypdf import PdfReader
import pymupdf
import io
from PIL import Image
from pathlib import Path

from sqlalchemy import create_engine

In [3]:
reader = PdfReader("data/ANISHKUMAR_BT21CS009_1733817042262.pdf")
print(len(reader.pages))

1


In [6]:
page = reader.pages[0]
text = page.extract_text()
print(text)

Registration ID BT21CS009
Name ANISH KUMAR SINGH
Programme Name Bachelor of Technology (Computer Science and Engineering)
Batch Year 2021
Status REGULAR
Semester Eighth
Registered Courses
Course Name Course Code Credit Date And Time
Major Project SOE-B-CSE-21-F02 10.0 10-12-2024 01:19 PM
Distributed SystemSOE-B-CSE-21-
F03(2)3.0 10-12-2024 01:19 PM
Soft Computing-2SOE-B-CSE-21-
F04(1)3.0 10-12-2024 01:19 PM
Natural Language ProcessingSOE-B-CSE-21-
F05(3)3.0 10-12-2024 01:19 PM
Soft Computing-2 LabSOE-B-CSE-21-
F06(1)1.0 10-12-2024 01:19 PM
Natural Language Processing LabSOE-B-CSE-21-
F07(3)1.0 10-12-2024 01:19 PM
(Min: 1.0 Max: 26.0) Total 21.0
Generated on 10-12-2024 01:20 PM


In [7]:
with open("output.txt", "w") as f:
    for word in text:
        f.write(word)

In [11]:
for count, image_file_object in enumerate(page.images):
    with open(str(count) + image_file_object.name, "wb") as fp:
        fp.write(image_file_object.data)

In [36]:
def extract_text_image(pdf):
    page_no = 1
    for page in pdf.pages:
        text = page.extract_text()
        with open(f"extracted_text/result_page_{page_no}.txt", "w") as f:
            for word in text:
                f.write(word)
        page_no += 1
        
        try:
            for count, image_file_object in enumerate(page.images):
                with open(f"extracted_images/{str(count) + image_file_object.name}", "wb") as fp:
                    fp.write(image_file_object.data)
        except Exception:
            continue

In [37]:
reader = PdfReader("data/xii_ch_1.pdf")
extract_text_image(reader)

In [40]:
doc = pymupdf.open("data/xii_ch_1.pdf")

In [ ]:
df = pd.DataFrame({"image": [], "string": []})
for page in doc:
    text = page.get_text()
    with open(f"extracted_text_pymupdf/result_page_{page.number}.txt", "w") as f:
            f.write(text)

    for image_index, img in enumerate(page.get_images(), start=1):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        with open(f"extracted_images_pymupdf/image{page.number+1}_{image_index}.{image_ext}", "wb") as image:
            image.write(image_bytes)

In [75]:
def upload(df, table):
    engine = create_engine(postgres_url)
    df.to_sql(table, engine, if_exists="replace", index=False)

In [83]:
df = pd.DataFrame({"page_no": [], "image": [], "string": []})
for page in doc:
    page_no = page.number+1
    has_img = len(page.get_images())
    if has_img > 2:
        has_img = "True"
    else:
        has_img = "False"
    text = page.get_text()
    df = pd.concat([df, pd.DataFrame({"page_no": [page_no], "image": [has_img], "string": [text]})], ignore_index=True)

df.to_csv("xii_ch1.csv", index=False)

In [84]:
upload(df, "xii_ch1")